# Predicting Diabetes with Logistic Regression

### We’ll be using Machine Learning to predict whether a person has diabetes or not, based on information about the patient such as blood pressure, body mass index (BMI), age, etc. 

- We will import pandas to read our data from a CSV file and manipulate it for further use. 
- We will use numpy to convert out data into a format suitable to feed our classification model. 
- We’ll use seaborn and matplotlib for visualizations.
- We will then import Logistic Regression algorithm from sklearn. This algorithm will help us build our classification model.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


## Dataset information:

- This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases.
Several constraints were placed on the selection of these instances from a larger database. 
In particular, all patients here are females at least 21 years old of Pima Indian heritage.


- Pregnancies: Number of times pregnant

- Glucose: Plasma glucose concentration a 2 hours in an oral glucose tolerance test

- BloodPressure: Diastolic blood pressure (mm Hg)

- SkinThickness: Triceps skin fold thickness (mm)

- Insulin: 2-Hour serum insulin (mu U/ml)

- BMI: Body mass index (weight in kg/(height in m)^2)

- DiabetesPedigreeFunction: Diabetes pedigree function (a function which scores likelihood of diabetes based on family history)

- Age: Age (years)

- Outcome: Class variable (0 or 1)



### Is our data is clean (has no null values, etc)?

In [ ]:
diabetesDF = pd.read_csv('assets/diabetes.csv')
diabetesDF.head()

In [ ]:
diabetesDF.info()

### The data does have some missing values (see Insulin = 0) in the samples in the previous figure. Ideally we could replace these 0 values with the mean value (impute) for that feature, but we’ll skip that for now.

## Data Exploration

Let us now explore our data set to get a feel of what it looks like and get some insights about it.
Let’s start by finding correlation of every pair of features (and the outcome variable), and visualize the correlations using a heatmap.  In the heatmap, brighter colors indicate more correlation.

In [ ]:
corr = diabetesDF.corr()
corr

### Glucose levels, age, BMI and number of pregnancies all have significant correlation with the outcome variable

In [ ]:
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)

Let’s also look at how many people in the dataset are diabetic and how many are not.

In [ ]:
sns.countplot(x = 'Outcome', data = diabetesDF, palette = 'magma')

It is also helpful to visualize relations between a single variable and the outcome. Below, we’ll see the relation between age and outcome.

In [ ]:
sns.barplot(x = 'Outcome', y = 'Age',  data = diabetesDF,
            palette = 'hls',
            order = [0, 1],  
            capsize = 0.05,             
            saturation = 8,             
            errcolor = 'gray', errwidth = 2,  
            ci = 'sd'   
            )

## Dataset Preparation (splitting and normalization)

When using machine learning algorithms we should always split our data into a training set and test set. We will also separate out some data for manual cross checking.

The data set consists of record of 768 patients in total. To train our model we will be using 650 records. We will be using 100 records for testing, and the last 18 records to cross check our model.

In [ ]:
#Total 768 patients record
#Using 650 data for training
# Using 100 data for testing
#Using 18 data for checking

dfTrain = diabetesDF[:650]
dfTest = diabetesDF[650:750]
dfCheck = diabetesDF[750:]

Next, we separate the label and features (for both training and test dataset). In addition to that, we will also convert them into NumPy arrays as our machine learning algorithm process data in NumPy array format.

In [ ]:
#Separating label and features and converting to numpy array to feed into our model
trainLabel = np.asarray(dfTrain['Outcome'])
trainData = np.asarray(dfTrain.drop('Outcome',axis = 1))
testLabel = np.asarray(dfTest['Outcome'])
testData = np.asarray(dfTest.drop('Outcome', axis = 1))


### Data Normalization

As the final step before using machine learning, we will normalize our inputs. Machine Learning models often benefit substantially from input normalization. It also makes it easier for us to understand the importance of each feature later, when we’ll be looking at the model weights. We’ll normalize the data such that each variable has 0 mean and standard deviation of 1.

In [ ]:
# Normalize the data 
means = np.mean(trainData, axis=0)
stds = np.std(trainData, axis=0)
print(means)
print(stds)


trainData = (trainData - means)/stds
testData = (testData - means)/stds

#means = np.mean(trainData, axis=0)
#stds = np.std(trainData, axis=0)
#print(means)
#print(stds)
# stds = np.std(trainData, axis=0)

## We will use the our training data (trainData) to create a Logistic Regression.

In [ ]:
diabetesCheck = LogisticRegression()
diabetesCheck.fit(trainData,trainLabel)
accuracy = diabetesCheck.score(testData,testLabel)
print("accuracy = ",accuracy * 100,"%")

## Interpreting the Model

To get a better sense of what is going on inside the logistic regression model, we can visualize how our model uses the different features and which features have greater effect.

In [ ]:
coeff = list(diabetesCheck.coef_[0])
coeff

In [ ]:
labels = list(dfTrain.drop('Outcome',axis = 1).columns)
labels

In [ ]:
features = pd.DataFrame()
features['Features'] = labels
features['importance'] = coeff
features.sort_values(by=['importance'], ascending=True, inplace=True)
features['positive'] = features['importance'] > 0
features.set_index('Features', inplace=True)
features.importance.plot(kind='barh', figsize=(11, 6),color = features.positive.map({True: 'blue', False: 'red'}))
plt.xlabel('Importance')

### From the above figure, we can draw the following conclusions.

- Glucose level, BMI, pregnancies and diabetes pedigree function have significant influence on the model, specially glucose level and BMI. It is good to see our machine learning model match what we have been hearing from doctors our entire lives!

- Blood pressure has a negative influence on the prediction, i.e. higher blood pressure is correlated with a person not being diabetic. (also, note that blood pressure is more important as a feature than age, because the magnitude is higher for blood pressure).

- Although age was more correlated than BMI to the output variables (as we saw during data exploration), the model relies more on BMI. This can happen for several reasons, including the fact that the correlation captured by age is also captured by some other variable, whereas the information captured by BMI is not captured by other variables.

Note that this above interpretations require that our input data is normalized. Without that, we can’t claim that importance is proportional to weights.

In [ ]:
dfCheck.head()

In [ ]:
sampleData = dfCheck[:5]
sampleDataFeatures = np.asarray(sampleData.drop('Outcome',axis = 1))
sampleDataFeatures

In [ ]:
# predict Using the testData set

prediction = diabetesCheck.predict(testData)

print('prediction:', prediction)

## Measuring Model Performance

In [ ]:
# Use score method to get accuracy of model
score = diabetesCheck.score(testData, testLabel)
print(score)

## Next steps

There are lots of ways to improve the above model. Here are some ideas.

- Input feature bucketing should help, i.e. create new variables for blood pressure in a particular range, glucose levels in a particular range, and so on.
- You could also improve the data cleaning, by replacing 0 values with the mean value.
- Read a bit about what metrics do doctors rely on the most to diagnose a diabetic patient, and create new features accordingly.